# Импорт библиотек

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Загрузка и предобработка данных

In [ ]:
# Укажите путь к вашим данным
data_dir = 'stanford dogs dataset/images/Images'

# Настройки генератора изображений
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    horizontal_flip=True,
    rotation_range=20,
    zoom_range=0.2
)

# Генератор тренировочных данных
train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

# Генератор валидационных данных
val_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

# Создание и компиляция модели

In [ ]:
# Загрузка предобученной модели ResNet50 без верхнего слоя
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Создание модели
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(256, activation='relu'),
    Dense(len(train_generator.class_indices), activation='softmax')
])

# Компиляция модели
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Обучение модели

In [ ]:
# Настройки для сохранения лучшей модели и ранней остановки
checkpoint = ModelCheckpoint('best_model.keras', monitor='val_loss', save_best_only=True, mode='min')
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Обучение модели
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=50,
    callbacks=[checkpoint, early_stop]
)

Оценка модели

In [ ]:
# Графики точности и потерь
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='train_accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.legend()
plt.title('Accuracy')

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='train_loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.legend()
plt.title('Loss')
plt.show()

# Тестирование модели на новых данных

In [ ]:
# Пример тестирования модели на новых данных
test_image_path = 'Спаниель.png'
test_image = tf.keras.preprocessing.image.load_img(test_image_path, target_size=(224, 224))
test_image = tf.keras.preprocessing.image.img_to_array(test_image) / 255.0
test_image = np.expand_dims(test_image, axis=0)

# Предсказание породы
prediction = model.predict(test_image)
predicted_class = np.argmax(prediction, axis=1)
print(f'Predicted class: {predicted_class}')